In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta
import random

In [2]:
#FUNCIONES:

def conexion():
    db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "2JTovldO",
                             database = None)
    cursor = db.cursor()
    return db, cursor

def cerrar_conexion(cursor, db):
    cursor.fetchall()
    cursor.close()
    db.close()

def crear_database(database):
    db, cursor = conexion()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
    cerrar_conexion(cursor, db)

def crear_tablas(query, database):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)
    cursor.execute(query)
    cerrar_conexion(cursor, db)

def columnas_tabla(database, tabla):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)

    # Obtener las columnas de la tabla
    query_columns = f"DESCRIBE {tabla};"
    cursor.execute(query_columns)

    # En la función DESCRIBE, la columna que indica si una columna es una clave primaria o no se encuentra en la posición 3 de la tupla devuelta. La estructura de la tupla que se obtiene al ejecutar DESCRIBE es la siguiente:
    #Nombre de la columna
    #Tipo de dato de la columna
    #Indicador de nulidad ("YES" si puede ser nulo, "NO" si no puede ser nulo)
    #Key (indicador de clave primaria, "PRI" si es clave primaria)
    cursor.execute(f"SELECT * FROM {tabla} LIMIT 0;")
    columnas = cursor.column_names[1:]
    cursor.fetchall()
    
    db.commit()
    cerrar_conexion(cursor, db)
    return columnas

def insertar_datos(database, tabla, diccionario):
    db, cursor = conexion()
    
    query_database = f"USE {database};"
    cursor.execute(query_database)
    cantidad_registros = len(list(diccionario.values())[0])
    for i in range(cantidad_registros):
        # Construye la parte de VALUES con los valores correspondientes para cada llave
        values_part = ', '.join([f"%s" for columna, valores in diccionario.items()])
        query = f"INSERT INTO {tabla} ({', '.join(diccionario.keys())}) VALUES ({values_part});"
        cursor.execute(query, tuple(valores[i] for valores in diccionario.values()))
    
    db.commit()
    cerrar_conexion(cursor, db)

def seleccion_tabla(database, tabla, columnas, limit = None):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)

    query = f"SELECT {columnas} FROM {tabla} LIMIT {limit};"
    cursor.execute(query)
    resultados = cursor.fetchall()  # Recupera los resultados de la consulta
    cerrar_conexion(cursor, db)
    
    return resultados

def df_tablas(database, tabla):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)

    query = f"SELECT * FROM {tabla};"
    df = pd.read_sql_query(query, db)
    cerrar_conexion(cursor, db)
    return df

In [3]:
crear_database("HACK_A_BOSS")

In [4]:
query_modulos = """CREATE TABLE IF NOT EXISTS modulos (
                modulo_id INT UNSIGNED AUTO_INCREMENT NOT NULL,
                modulos   VARCHAR(25),
                PRIMARY KEY (modulo_id)
                );"""

crear_tablas(query_modulos, "HACK_A_BOSS")

columnas_modulos = columnas_tabla("HACK_A_BOSS", "modulos")
registros_modulo = ["Python", "Matemáticas", "Ciencia de Datos", "SQL", "Machine Learning", "PySpark", "Streamlit"]
data_modulos = [registros_modulo]
diccionario_modulos = {}
for keys, values in zip(columnas_modulos, data_modulos):
    diccionario_modulos[keys] = values

insertar_datos("HACK_A_BOSS", "modulos", diccionario_modulos)

In [5]:
query_bootcamps = """CREATE TABLE IF NOT EXISTS bootcamps (
                    bootcamp_id INT UNSIGNED AUTO_INCREMENT NOT NULL,
                    bootcamp VARCHAR(255),
                    inicio_bootcamp DATE,
                    final_bootcamp DATE,
                    PRIMARY KEY (bootcamp_id)
                    )"""

crear_tablas(query_bootcamps, "HACK_A_BOSS")

columnas_bootcamps = columnas_tabla("HACK_A_BOSS", "bootcamps")

registros_bootcamp = [f"DSB{i}RT" if i > 9 else f"DSB0{i}RT" for i in range(1, 201)]

registros_inicio_bootcamp = []
resgistros_final_bootcamp = []

for i in range(200):
    inicio = datetime(2020, 1, 1)
    patron = timedelta(days = 30*i)
    x = inicio + patron
    registros_inicio_bootcamp.append(x.strftime("%Y-%m-%d"))

for i in registros_inicio_bootcamp:
    inicio = datetime.strptime(i, "%Y-%m-%d")
    patron = timedelta(days = 154)
    x = inicio + patron
    resgistros_final_bootcamp.append(x.strftime("%Y-%m-%d"))

data_bootcamps = [registros_bootcamp, registros_inicio_bootcamp, resgistros_final_bootcamp]

diccionario_bootcamp = {}
for keys, values in zip(columnas_bootcamps, data_bootcamps):
    diccionario_bootcamp[keys] = values

insertar_datos("HACK_A_BOSS", "bootcamps", diccionario_bootcamp)

In [6]:
query_estudiantes = """CREATE TABLE IF NOT EXISTS estudiantes (
                    estudiante_id INT UNSIGNED AUTO_INCREMENT NOT NULL,
                    nombre VARCHAR(255),
                    apellido VARCHAR(255),
                    email VARCHAR(255) UNIQUE,
                    beca BOOL,
                    inscripcion DATE,
                    bootcamp_id INT UNSIGNED,
                    PRIMARY KEY (estudiante_id),
                    FOREIGN KEY (bootcamp_id) REFERENCES bootcamps (bootcamp_id)
                    );"""

crear_tablas(query_estudiantes, "hack_a_boss")

columnas_estudiantes = columnas_tabla("HACK_A_BOSS", "estudiantes")

registros_nombre = [f"nombre{i}" if i > 9 else f"nombre0{i}" for i in range(1, 7533)]
registros_apellido = [f"apellido{i}" if i > 9 else f"apellido0{i}" for i in range(1, 7533)]
registros_email = [f"{nombre}{apellido}@python.com" for nombre, apellido in zip(registros_nombre, registros_nombre)]
registros_beca = [random.choice([True, False]) for _ in range(7532)]
registros_inscripcion = [(datetime(2019, 1, 1) + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(7532)]
registros_bootcamp_id = [random.randint(1, 200) for _ in range(7532)]

data_estudiantes = [registros_nombre, registros_apellido, registros_email, registros_beca, registros_inscripcion, registros_bootcamp_id]

diccionario_estudiantes = {}
for keys, values in zip(columnas_estudiantes, data_estudiantes):
    diccionario_estudiantes[keys] = values

insertar_datos("HACK_A_BOSS", "estudiantes", diccionario_estudiantes)

In [7]:
query_modulo_bootcamp = """CREATE TABLE IF NOT EXISTS modulo_bootcamp (
                        bootcamp_id INT UNSIGNED,
                        modulo_id INT UNSIGNED,
                        puntuacion INT,
                        PRIMARY KEY (bootcamp_id, modulo_id),
                        FOREIGN KEY (bootcamp_id) REFERENCES bootcamps (bootcamp_id),
                        FOREIGN KEY (modulo_id) REFERENCES modulos (modulo_id)
                        );"""

crear_tablas(query_modulo_bootcamp, "hack_a_boss")

columnas_modulo_bootcamp = ["bootcamp_id", "modulo_id", "puntuacion"]

registros_mb_bootcamp_id =  sorted([(i % 200) + 1 for i in range(1400)])
registros_mb_modulo_id = [(i % 7) + 1 for i in range(1400)]
registros_puntuacion = [random.randint(1, 10) for _ in range(1400)]

data_modulo_bootcamp = [registros_mb_bootcamp_id, registros_mb_modulo_id, registros_puntuacion]

diccionario_modulo_bootcamp = {}
for keys, values in zip(columnas_modulo_bootcamp, data_modulo_bootcamp):
    diccionario_modulo_bootcamp[keys] = values

insertar_datos("hack_a_boss", "modulo_bootcamp", diccionario_modulo_bootcamp)

In [6]:
query_asistencias = """CREATE TABLE IF NOT EXISTS asistencias (
                    asistencia_id INT UNSIGNED AUTO_INCREMENT NOT NULL,
                    estudiante_id INT UNSIGNED,
                    asistencia BOOL,
                    fecha DATE,
                    PRIMARY KEY (asistencia_id),
                    FOREIGN KEY (estudiante_id) REFERENCES estudiantes (estudiante_id)
                    );"""

crear_tablas(query_asistencias, "HACK_A_BOSS")

In [11]:
columnas_asistencias = columnas_tabla("HACK_A_BOSS", "asistencias")

registros_a_estudiante_id = sorted([(i % 7532) + 1 for i in range(1159928)])
registros_a_asistencia = [random.choice([True, False]) for _ in range(1159928)]

lista_bootcamp_id_estudiantes = diccionario_estudiantes["bootcamp_id"]
lista_inicio_bootcamp = diccionario_bootcamp["inicio_bootcamp"]
lista_bootcamp_id = [x for x in range(1,201)]

dic_bootid_inicio = {}
for keys, values in zip(lista_bootcamp_id, lista_inicio_bootcamp):
    dic_bootid_inicio[keys] = values

fechas_correspondientes = [dic_bootid_inicio[id] for id in lista_bootcamp_id_estudiantes]

# Número de fechas consecutivas a añadir después de cada fecha
num_fechas_consecutivas = 153

# Lista para almacenar el resultado final
registros_a_fecha = []

# Iterar sobre la lista fechas_correspondientes
for fecha in fechas_correspondientes:
    registros_a_fecha.append(fecha)  # Añadir el elemento actual

    # Añadir las 154 fechas consecutivas
    fecha_actual = datetime.strptime(fecha, '%Y-%m-%d')
    for _ in range(num_fechas_consecutivas):
        fecha_actual += timedelta(days=1)
        registros_a_fecha.append(fecha_actual.strftime('%Y-%m-%d'))

data_asistencias = [registros_a_estudiante_id, registros_a_asistencia, registros_a_fecha]
diccionario_asistencias = {}
for keys, values in zip(columnas_asistencias, data_asistencias):
    diccionario_asistencias[keys] = values

In [12]:
insertar_datos("HACK_A_BOSS", "asistencias", diccionario_asistencias)

In [15]:
seleccion_tabla("hack_a_boss", "asistencias", "*", 3)

[(1, 1, 1, datetime.date(2030, 8, 6)),
 (2, 1, 0, datetime.date(2030, 8, 7)),
 (3, 1, 0, datetime.date(2030, 8, 8))]

In [5]:
# TRAERME TABLAS:
df_modulos = df_tablas("hack_a_boss", "modulos")
df_modulo_bootcamp = df_tablas("hack_a_boss", "modulo_bootcamp")
df_bootcamps = df_tablas("hack_a_boss", "bootcamps")
df_estudiantes = df_tablas("hack_a_boss", "estudiantes")
df_asistencias = df_tablas("hack_a_boss", "asistencias")

C:\Users\Ane Zubieta\AppData\Local\Temp\ipykernel_17944\923749119.py:81: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, db)


In [6]:
df_asistencias

,asistencia_id,estudiante_id,asistencia,fecha
0,1,1,1,2030-08-06
1,2,1,0,2030-08-07
2,3,1,0,2030-08-08
3,4,1,1,2030-08-09
4,5,1,0,2030-08-10
...,...,...,...,...
1159923,1159924,7532,1,2023-11-10
1159924,1159925,7532,0,2023-11-11
1159925,1159926,7532,0,2023-11-12
1159926,1159927,7532,1,2023-11-13
